Wedge vs Rubblestone Classification

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import pytorch_lightning as pl
from torch.optim import Adam, SGD, AdamW
from torch.optim.lr_scheduler import StepLR
import random
import numpy as np

from torch.utils.data import Dataset
import pandas as pd

random.seed(47)
np.random.seed(47)
torch.manual_seed(47)

In [2]:
augmented = True
normalize = True

method = "Yaser's method"
n_attributes = 12
K = 5

INPUT_LAYER = 48
INPUT_ATTRIBUTES_METHOD = "Yaser's method"

BATCH_SIZE = 16
EPOCHS = 16

if not augmented:
    print("augmented is False, ignoring normalize, method, n_attributes and K")

In [3]:

dataframes = []#[pd.read_excel("shuffled_data.ods", engine="odf")]
#dataframes[-1]["Id_stone_type"] = dataframes[-1]["Id_stone_type"].map(lambda s: 0 if s == "Wedge" else 1)

for k in range(K):
    if augmented:
        filename = "datasets/augmented m({}) a({}) k({}){}.ods".format(method, n_attributes, k + 1, (" N" if normalize else ""))
    else:
        filename = "datasets/k({}).ods".format(k + 1)
        
    dataframes.append(pd.read_excel(filename, engine="odf"))
    dataframes[-1]["Id_stone_type"] = dataframes[-1]["Id_stone_type"].map(lambda s: 0 if s == "Wedge" else 1)

"""
dataframe = pd.read_excel("data.ods", engine="odf")
wedge,rubble = 0,0
for type in dataframe["Id_stone_type"]:
    if type == "Wedge":
        wedge += 1
    elif type == "RubbleStone":
        rubble += 1
print(wedge, rubble)
dataframe["Id_stone_type"] = dataframe["Id_stone_type"].map(lambda s: 0 if s == "Wedge" else 1)
dataframe.head(5)
"""

pd.read_excel("shuffled_data.ods", engine="odf").head(5)



,unnamed.1,Id_wall_facing_element,Id_stone_type,Vertices,Faces,Area,Convex_hull_volume,Oriented_bounding_box_volume,Center_x,Center_y,...,Flakiness,Sphericity,Mean_discrete_gaussian_curvature,Var_discrete_gaussian_curvature,Azimuth_longest,Elevation_longest,Azimuth_middle,Elevation_middle,Azimuth_shortest,Elevation_shortest
0,880,Phaistos_UM1005_Socle_31_Wedge,Wedge,62,96,0.007876,0.000092,0.000173,0.498011,0.343761,...,0.864475,0.588857,2.532808,6.070731,65.532211,12.195591,8.613949,72.048735,26.071202,19.187490
1,122,Anavlochos Demargne terrace_Elevation_244_Wedge,Wedge,69,114,0.006150,0.000075,0.000180,-3.016569,0.201758,...,0.543187,0.760899,1.390594,2.984991,38.704729,62.962013,5.777196,28.887084,75.694453,47.553249
2,968,Phaistos_UM901_MM IIIB Foundation_14_RubbleStone,RubbleStone,289,513,0.007952,0.000093,0.000231,0.495457,-0.518704,...,0.819860,0.610814,3.576002,21.155149,17.761279,62.909181,7.522540,28.064784,80.218623,23.697364
3,734,Hagia_Triada_Muro_Laviosa_I Phase_45_Wedge,Wedge,258,467,0.013649,0.000240,0.000616,3.626062,-0.074495,...,0.606624,0.739493,2.331775,11.976726,7.037591,10.574681,14.031368,79.096638,83.694963,32.142293
4,386,\Anavlochos Quartier de La forge Wall AN19-04-...,Wedge,601,1101,0.014498,0.000205,0.000470,-2.314442,-0.034493,...,0.845019,0.582354,5.887830,49.315872,88.580295,89.059055,6.619356,3.461018,81.117584,76.780028


In [4]:
TOP_ATTRIBUTES = {"random" : random.choices(list(range(3, 51)), k = 48),
           "average threshold" :  [37, 7, 39, 5, 6, 25, 26, 13, 23, 3, 38, 4, 44, 21, 22, 11, 24, 20, 43, 36, 12, 9, 40, 19, 33, 48, 49, 18, 32, 41, 35, 50, 45, 15, 27, 14, 28, 34, 47, 30, 42, 29, 8, 46, 17, 10, 31, 16], 
           "maximizer threshold": [37, 5, 7, 39, 6, 36, 3, 4, 38, 25, 21, 26, 23, 24, 20, 22, 44, 43, 12, 13, 17, 18, 27, 28, 29, 32, 33, 34, 35, 40, 46, 48, 49, 50, 11, 41, 8, 9, 10, 14, 15, 16, 19, 30, 31, 42, 45, 47],
           "Yaser's method" :     [37, 9, 40, 4, 3, 48, 8, 23, 17, 32, 25, 18, 29, 5, 11, 38, 13, 50, 16, 10, 26, 6, 21, 7, 24, 46, 28, 19, 39, 43, 36, 14, 15, 22, 34, 42, 33, 45, 44, 31, 20, 47, 30, 49, 41, 35, 27, 12],
           "chi2 n2" :            [3, 4, 5, 6, 7, 39, 44, 43, 37, 20, 36, 24, 11, 21, 25, 26, 22, 23, 13, 12, 41, 38, 9, 42, 15, 18, 27, 49, 45, 10, 46, 48, 17, 16, 31, 34, 35, 30, 47, 40, 50, 14, 19, 32, 28, 29, 33, 8],
           "chi2 n4" :            [44, 6, 7, 39, 5, 3, 4, 43, 36, 37, 9, 20, 25, 22, 12, 24, 13, 11, 23, 21, 26, 38, 41, 10, 49, 15, 27, 16, 42, 8, 45, 18, 19, 14, 17, 46, 34, 28, 33, 31, 35, 48, 40, 30, 32, 29, 50, 47],     
           "chi2 n8" :            [44, 6, 7, 39, 4, 3, 5, 43, 9, 36, 37, 20, 25, 12, 22, 21, 13, 24, 11, 23, 26, 38, 27, 41, 15, 49, 10, 34, 14, 16, 8, 45, 42, 33, 28, 19, 18, 17, 32, 46, 31, 35, 48, 40, 29, 30, 50, 47],
           "chi2 n16" :           [44, 6, 7, 39, 4, 3, 5, 43, 9, 37, 36, 24, 20, 25, 13, 22, 11, 12, 21, 23, 26, 27, 38, 41, 34, 49, 10, 14, 8, 15, 16, 42, 45, 32, 33, 28, 18, 19, 17, 46, 31, 35, 48, 29, 40, 30, 50, 47],
           "chi2 n32" :           [44, 6, 7, 39, 4, 3, 5, 43, 9, 37, 36, 12, 24, 20, 22, 13, 25, 11, 27, 21, 23, 26, 38, 34, 41, 8, 49, 10, 14, 15, 42, 16, 32, 45, 33, 28, 19, 18, 17, 46, 31, 35, 29, 48, 30, 40, 50, 47],
           "chi2 n64" :           [44, 6, 7, 39, 4, 3, 5, 9, 43, 37, 36, 12, 24, 20, 22, 25, 27, 13, 11, 21, 23, 26, 38, 34, 41, 8, 49, 10, 15, 14, 42, 16, 32, 45, 33, 28, 18, 19, 46, 17, 31, 35, 29, 30, 48, 40, 50, 47]
}

class StoneDataset(Dataset): #impolementare len e get item
    def __init__(self, dataframe, train = True, train_size = None, transforms = None, k = 0):
        super(Dataset, self).__init__()
        #train_size = round(train_size * len(dataframe))
        self.transforms = transforms
        #top_attributes = [37, 7, 39, 5, 6, 25, 26, 13, 23, 3, 38, 4] #trovati con tresh = average
        #top_attributes = [37, 5, 7, 39, 6, 36, 3, 4, 38, 25, 21, 26] #trovati con ricerca di tresh
        
        if train:
            #self.data = dataframe.iloc[:train_size, 3:]      #all attributes
            self.data = dataframe.iloc[:train_size, TOP_ATTRIBUTES[INPUT_ATTRIBUTES_METHOD][:INPUT_LAYER]]
            _targets = dataframe.iloc[:train_size, 2]
            
            #self.data = dataframe.iloc[:train_size, top_attributes]
            #print(self.data)
            _targets = dataframe.iloc[:train_size, 2]
        else:
            self.data = dataframe.iloc[train_size:, 3:]      #all attributes
            self.data = dataframe.iloc[train_size:, TOP_ATTRIBUTES[INPUT_ATTRIBUTES_METHOD][:INPUT_LAYER]]
            _targets = dataframe.iloc[train_size:, 2]

            #self.data = dataframe.iloc[train_size:, top_attributes]
            #_targets = dataframe.iloc[train_size:, 2]
        
        self.data = self.data.to_numpy()
        #self.targets = np.zeros((len(self.data), 2))
        #self.targets[:, 0] = _targets
        #self.targets[:, 1] = 1 - self.targets[:, 0]

        #print(self.targets)
        
        # to tensor
        self.data = torch.tensor(self.data, dtype = torch.float32)
        print("_targets.to_numpy:", _targets.to_numpy())
        self.targets = torch.stack(tuple([torch.tensor(_targets.to_numpy(), dtype = torch.float32)]))
        
        #self.targets = torch.tensor(self.targets, dtype = torch.float32).T

        print(self.targets.shape)

        #if transforms != None: TODO

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, i):
        if self.transforms:
            return self.transforms(self.data[i]), self.targets[:,i]
        return self.data[i], self.targets[:,i]


In [5]:
#network class
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.linear1 = nn.Linear(INPUT_LAYER, 128)
        self.linear2 = nn.Linear(128, 64)
        self.linear3 = nn.Linear(64 + INPUT_LAYER, 1) # 64 + INPUT_LAYER for skip connection

        self.dp1 = nn.Dropout(p = 0.5)
        self.dp2 = nn.Dropout(p = 0.4)

    def forward(self, inp):
        x = self.linear1(inp)
        x = F.leaky_relu(x, negative_slope = 0.05)  #F.tanh
        x = self.dp1(x)

        x = self.linear2(x)
        x = F.leaky_relu(x, negative_slope = 0.05)
        x = self.dp2(x)

        x = self.linear3(torch.cat([x, inp], dim = 1)) #skip connection
        #x = self.linear3(x)

        return x

In [6]:
#training class
class ClassifierLightning(pl.LightningModule):
    def __init__(self):
        super(ClassifierLightning, self).__init__()
        self.model = Classifier()
        self.criterion = nn.BCEWithLogitsLoss()

    def accuracy(self, y_hat, y): #y = expected
        return torch.where(((y_hat>0) == y),1.0,0.0).sum()/BATCH_SIZE
        y_hat = torch.where(F.sigmoid(y_hat) > 0.5, 1.0, 0.0)
        return torch.sum(torch.argmax(y_hat, dim = 0) == y).item()/y.shape[0]
    
        
    def forward(self, x):
        return self.model(x)
    
    def configure_optimizers(self):
        optimizer = AdamW(params = self.parameters(), lr = 5e-4, weight_decay = 3e-5)  #lr = 5e-4 * 1.2 con scheduler else 3e-4
        scheduler = StepLR(optimizer, step_size=32, gamma=0.8)
        #optimizer = SGD(params = self.parameters(), lr = 0.02, momentum = 0.9, weight_decay = 1e-5) #nesterov
        return [optimizer], [scheduler] #usare se c'è lo scheduler
        return optimizer                #altrimenti

    def training_step(self, train_batch, batch_id):
        self.model.train()
        x, y = train_batch
        logits = self(x)
        loss = self.criterion(logits, y)
        accuracy = self.accuracy(logits, y)
        self.log("train_accuracy", accuracy)
        self.log("train_loss", loss)
        return loss
    
    def validation_step(self, val_batch, batch_id):
        self.model.eval()
        x, y = val_batch
        logits = self(x)
        loss = self.criterion(logits, y)
        accuracy = self.accuracy(logits, y)
        self.log("val_accuracy", accuracy)
        self.log("val_loss", loss)
        return loss

In [7]:
print("gpu available:", torch.cuda.is_available())
print("device count:", torch.cuda.device_count())

gpu available: True
device count: 1


In [8]:
from torch.utils.data import DataLoader
from torchvision.transforms import Lambda

transform = Lambda(lambda x: x + torch.randn_like(x)*0.04)
for k in range(K):
    #creating train & validation sets
    train_set = StoneDataset(dataframes[k], train = True, train_size = len(dataframes[k]) - 813//K, transforms = transform) #813 = len of original dataset
    val_set = StoneDataset(dataframes[k], train = False, train_size = len(dataframes[k]) - 813//K)

    #loading sets
    train_loader = DataLoader(dataset = train_set, batch_size = BATCH_SIZE, shuffle = True)
    val_loader = DataLoader(dataset = val_set, batch_size = BATCH_SIZE, shuffle = False)

    #prendere un numero di feature a caso e azzerarle (?)

    print("train size:", len(train_set))
    print("val size:", len(val_set))

    name = "augmented m({}) a({}) k({}){} i({}) im({})".format(method, n_attributes, k + 1, (" N" if normalize else ""), INPUT_LAYER, INPUT_ATTRIBUTES_METHOD) if augmented else "k({}) i({}) im({})".format(k + 1, INPUT_LAYER, INPUT_ATTRIBUTES_METHOD)

    logger = pl.loggers.TensorBoardLogger("logs", name = ("augmented m({}) a({}) k({}){} i({}) im({})".format(method, n_attributes, k + 1, (" N" if normalize else ""), INPUT_LAYER, INPUT_ATTRIBUTES_METHOD) if augmented else "k({}) i({}) im({})".format(k + 1, INPUT_LAYER, INPUT_ATTRIBUTES_METHOD)))#, version = ("augmented m({}) a({}) k({}){}".format(method, n_attributes, k + 1, (" N" if normalize else "")) if augmented else "k({}) i({}) im({})".format(k + 1, INPUT_LAYER, INPUT_ATTRIBUTES_METHOD)) )
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    trainer = pl.Trainer(max_epochs = EPOCHS, logger = logger, accelerator = "gpu", devices = 1)
    model = ClassifierLightning().to(device)
    trainer.fit(model, train_loader, val_loader)

    #save model
    torch.save(model.state_dict(), "models/" + name)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


_targets.to_numpy: [0 1 1 ... 0 1 1]
torch.Size([1, 1949])
_targets.to_numpy: [1 1 1 0 0 0 0 0 1 1 1 1 0 0 0 0 0 1 1 1 0 0 1 1 0 1 1 0 1 1 0 1 0 0 1 1 0
 1 1 0 1 1 0 0 0 1 1 1 0 1 0 1 1 0 0 1 0 0 1 0 1 0 0 0 1 0 0 0 1 0 1 1 0 1
 1 1 0 1 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 1 1 1 1 1 0 1 1 1 1 1 0
 1 0 0 1 0 0 0 1 0 0 1 0 1 1 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0 1 1 0 0 1 0
 1 0 0 1 0 0 1 0 1 0 0 1 1 1]
torch.Size([1, 162])
train size: 1949
val size: 162


You are using a CUDA device ('NVIDIA GeForce RTX 3060') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type              | Params
------------------------------------------------
0 | model     | Classifier        | 14.6 K
1 | criterion | BCEWithLogitsLoss | 0     
------------------------------------------------
14.6 K    Trainable params
0         Non-trainable params
14.6 K    Total params
0.059     Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

c:\Users\PC\AppData\Local\Programs\Python\Python39\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


...


_targets.to_numpy: [1 1 1 ... 0 1 1]
torch.Size([1, 1949])
_targets.to_numpy: [1 0 1 1 1 0 0 1 1 1 1 1 1 0 0 0 1 1 1 1 0 1 0 1 1 1 1 0 1 0 0 1 1 0 1 1 0
 0 1 1 0 0 0 0 1 1 0 1 1 1 1 0 0 0 0 0 0 1 1 0 1 0 0 0 1 0 1 0 0 1 0 1 1 1
 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 0 0 0 1 0 1 0 1 0 0 0 1 0 0 0 1
 0 1 1 0 1 1 1 0 1 1 1 0 0 0 0 1 1 1 0 1 1 0 1 0 0 0 0 1 0 1 0 0 1 0 1 1 0
 0 1 1 0 1 1 0 0 0 1 0 1 0 0]
torch.Size([1, 162])
train size: 1949
val size: 162


TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3060') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type              | Params
------------------------------------------------
0 | model     | Classifier        | 14.6 K
1 | criterion | BCEWithLogitsLoss | 0     
------------------------------------------------
14.6 K    Trainable params
0         Non-trainable params
14.6 K    Total params
0.059     Total estimated model params size (MB)


Epoch 15: 100%|██████████| 122/122 [00:00<00:00, 200.94it/s, v_num=1]       

`Trainer.fit` stopped: `max_epochs=16` reached.


Epoch 15: 100%|██████████| 122/122 [00:00<00:00, 198.98it/s, v_num=1]


In [9]:
with torch.no_grad():
    input = torch.stack([val_set[i][0] for i in range(64)])
    print(model(input))

tensor([[ 2.9664e+00],
        [-4.1717e+00],
        [-5.2214e-01],
        [ 3.0706e+00],
        [ 1.2139e+00],
        [-3.5771e-01],
        [-2.1183e+00],
        [ 2.6912e+00],
        [-1.2913e-01],
        [ 8.7450e+00],
        [ 4.9046e-01],
        [-3.3906e+00],
        [ 6.9685e+00],
        [-2.5793e+00],
        [-8.5209e-01],
        [ 3.6390e-01],
        [ 5.1519e+00],
        [ 2.0386e+00],
        [-1.0543e+00],
        [ 3.1870e+00],
        [-2.4056e+00],
        [ 5.0588e+00],
        [-8.9132e-01],
        [ 4.7143e+00],
        [ 1.2652e+00],
        [-5.3037e-01],
        [-2.1218e+00],
        [-4.0711e+00],
        [ 1.5076e+00],
        [-3.8185e+00],
        [-1.3815e+00],
        [ 5.9051e+00],
        [ 3.3784e+00],
        [-5.2965e+00],
        [ 6.6588e+00],
        [-7.2060e-01],
        [ 2.7523e-01],
        [-1.6632e+00],
        [ 1.9011e+01],
        [ 1.1300e+00],
        [ 2.4853e+00],
        [-4.1008e+00],
        [-4.8939e+00],
        [-4

In [10]:
with torch.no_grad():
    model.eval()
    acc = 0
    for x,y in iter(val_loader):
        acc += torch.where(((model(x)>0) == y),1.0,0.0).sum()
    print(acc/len(val_set))

    acc = 0
    for i in range(len(val_set)):
        #print(model(val_set[i][0]), val_set[i][1])
        #if (model( torch.stack([val_set[i][0]]) ) > 0) == val_set[i][1]:  #skip connection
        if (model(val_set[i][0]) > 0) == val_set[i][1]: 
            acc += 1
    print(acc/len(val_set))

    acc = 0
    for i in range(len(train_set)):
        #print(model(val_set[i][0]), val_set[i][1])
        #if (model( torch.stack([train_set[i][0]]) ) > 0) == train_set[i][1]: #skip connection
        if (model(train_set[i][0]) > 0) == train_set[i][1]: 
            acc += 1
    print("train acc", acc/len(train_set))

tensor(0.7901)


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)